# Model Experiments for Skin Cancer Detection

**Team**: Dr. Homi Jehangir Bhabha  
**Problem Statement**: PS 18

This notebook experiments with different model architectures and hyperparameters.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Add src to path
sys.path.insert(0, '../src')

from config import *
from model import create_model
from dataset import create_data_generators

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## 1. Load Data

In [ ]:
# Create data generators
if os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR):
    train_generator, val_generator = create_data_generators(
        TRAIN_DIR, VAL_DIR, BATCH_SIZE, IMAGE_SIZE
    )
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {val_generator.samples}")
    print(f"Classes: {train_generator.class_indices}")
else:
    print("Training data not found. Please add data to data/raw/train/ and data/raw/val/")

## 2. Experiment with Different Architectures

In [ ]:
# Compare model architectures
architectures = ['efficientnetb0', 'resnet50', 'vgg16', 'mobilenet']

for arch in architectures:
    print(f"\n{'='*50}")
    print(f"Architecture: {arch.upper()}")
    print(f"{'='*50}")
    
    try:
        model = create_model(arch, NUM_CLASSES, IMAGE_SIZE + (3,))
        model.summary()
        
        # Count parameters
        total_params = model.count_params()
        trainable_params = sum([tf.size(w).numpy() for w in model.trainable_weights])
        
        print(f"\nTotal parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")
        print(f"Non-trainable parameters: {total_params - trainable_params:,}")
    except Exception as e:
        print(f"Error creating model: {e}")

## 3. Visualize Model Architecture

In [ ]:
# Create and visualize a model
model = create_model(MODEL_NAME, NUM_CLASSES, IMAGE_SIZE + (3,))

# Plot model architecture (requires pydot and graphviz)
try:
    from tensorflow.keras.utils import plot_model
    plot_model(model, to_file='model_architecture.png', 
               show_shapes=True, show_layer_names=True)
    print("Model architecture saved to model_architecture.png")
except Exception as e:
    print(f"Could not plot model: {e}")
    print("Install graphviz: apt-get install graphviz")
    print("Install pydot: pip install pydot")

## 4. Test Data Augmentation

In [ ]:
# Visualize data augmentation effects
if os.path.exists(TRAIN_DIR):
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    
    # Create augmentation generator
    aug_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=ROTATION_RANGE,
        width_shift_range=WIDTH_SHIFT_RANGE,
        height_shift_range=HEIGHT_SHIFT_RANGE,
        horizontal_flip=HORIZONTAL_FLIP,
        zoom_range=ZOOM_RANGE
    )
    
    # Generate augmented images
    aug_generator = aug_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=IMAGE_SIZE,
        batch_size=1,
        class_mode='categorical'
    )
    
    # Display augmented versions of one image
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.ravel()
    
    for i in range(8):
        img, label = next(aug_generator)
        axes[i].imshow(img[0])
        axes[i].axis('off')
        axes[i].set_title(f'Augmented {i+1}')
    
    plt.suptitle('Data Augmentation Examples', fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("Training data not found")

## 5. Quick Training Test

In [ ]:
# Quick training test with few epochs
if os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR):
    print("Running quick training test (5 epochs)...")
    
    model = create_model(MODEL_NAME, NUM_CLASSES, IMAGE_SIZE + (3,))
    
    history = model.fit(
        train_generator,
        epochs=5,
        validation_data=val_generator,
        verbose=1
    )
    
    # Plot training history
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train')
    axes[0].plot(history.history['val_accuracy'], label='Validation')
    axes[0].set_title('Model Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train')
    axes[1].plot(history.history['val_loss'], label='Validation')
    axes[1].set_title('Model Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print("Training data not found")

## 6. Grad-CAM Visualization (Optional)

In [ ]:
# Grad-CAM for model interpretability
# This shows which parts of the image the model focuses on

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate Grad-CAM heatmap"""
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

print("Grad-CAM function defined. Use this after training to visualize model attention.")

## 7. Summary

This notebook demonstrates:
- Different model architectures
- Model complexity comparison
- Data augmentation effects
- Quick training tests
- Visualization techniques

For full training, use:
```bash
python main.py train --model efficientnetb0 --epochs 50
```